1. import data from ACLED
2. Connect to google sheets
3. paste to google sheets
4.

In [11]:
import pandas as pd
import requests
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [49]:
r = requests.get("http://acleddata.com/api/acled/read")

In [50]:
data = r.json()

In [51]:
data = data['data']

In [52]:
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('ACLED_to_Google_Sheets-e244e52c2f43.json', scope)

In [53]:
gc = gspread.authorize(credentials)

In [55]:
sheet = gc.open('new_acled_sheet').sheet1

In [48]:
#for row in data:
#    sheet.append_row(row.values())

#### 1. Create a dataframe from data
2. Send columns to sheet
3. Send data to sheet

In [56]:
#create dataframe

df = pd.DataFrame(data)

In [60]:
columns = df.columns.values.tolist()
len(columns)

26

In [62]:
cell_list = sheet.range('A1:Z500')

In [67]:
for cell in cell_list:
    val = columns[cell.col-1]
    cell.value = val

In [70]:
sheet.update_cells(cell_list)

In [73]:
num_lines, num_columns = df.shape

In [74]:
cell_list = sheet.range('A2:Z501')
for cell in cell_list:
    val = df.iloc[cell.row-2, cell.col-1]
    if type(val) is str:
        val = val.decode('utf-8')
    cell.value = val
sheet.update_cells(cell_list)